<hr style="border:2px solid gray">

#**STEP: 1/5** - Load and explore the dataset.

### Import libraries and define the data path

In [1]:
# import the necessary libraries 

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [2]:
# Update the DATA_PATH variable

import sys

if 'google.colab' in sys.modules:
  # If you're on Colab:
  DATA_PATH = 'https://raw.githubusercontent.com/bloominstituteoftechnology/ds_code_along_unit_2/main/data/flight/'
else:
  # If you're working locally:
  DATA_PATH = '..../data/'

### Load the training dataset

In [3]:
# load your training set (we will be working with only this one dataset for this notebook)
df = pd.read_excel(DATA_PATH + 'Data_Train.xlsx')

### Explore the dataset

In [4]:
# print the first 5 rows
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [5]:
# Determine the shape
df.shape

(10683, 11)

In [6]:
# explore the datatypes of all the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


**Insight:** All features are categorical except 'Price' which is an integer.

In [7]:
# how many null values are there in the dataset
print('Null Values:', df.isnull().sum().sum())
df.isnull().sum()

Null Values: 2


Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [11]:
# Fetching those row(s) where there is/are missing values
df[df['Route'].isna() | df['Total_Stops'].isna()] # the | operator means 'or'

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price


**Insight:** The row at index 9039 contains missing values in both the Route and Total_Stops feature.

In [9]:
# Lets delete this row for now. In later modules we will learn how to 'impute' null values.
df.dropna(inplace = True)

In [10]:
# Check head and shape of dataframe to confirm deleted data
print(df.shape)
df.head()

(10682, 11)


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


<hr style="border:2px solid gray">

#**STEP: 2/5** - Clean the two selected features


### Clean the 'Duration' column

In [12]:
# Duration is in a string format. It must be converted into a numerical format. 
df['Duration']

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10682, dtype: object

In [13]:
# converting duration into minutes

def convert_duration(duration):
  if len(duration.split()) == 2:
    hours = int(duration.split()[0][:1]) # select the first number
    minutes = int(duration.split()[1][:-1]) # select the second number
    return hours * 60 + minutes
  else:
    return int(duration[:-1]) * 60
        
        
df['Duration'] = df['Duration'].apply(convert_duration)

In [14]:
'''
In case you're wondering how a split function works, this is an illustration.
split() will split the string based on a delimiter.
By default, the space character is the delimiter if none is provided.'''

x = '2h 50m'
x.split()

['2h', '50m']

In [15]:
x.split()[1][:-1]

'50'

### Clean the 'Total_Stops' column.

In [16]:
# Total_Stops is in string format. It must be converted into a numerical format. 
df['Total_Stops'].value_counts()

1 stop      5625
non-stop    3491
2 stops     1520
3 stops       45
4 stops        1
Name: Total_Stops, dtype: int64

In [17]:
df.describe()

,Duration,Price
count,10682.000000,10682.000000
mean,244.089122,9087.214567
std,248.136532,4611.548810
min,65.000000,1759.000000
25%,105.000000,5277.000000
50%,160.000000,8372.000000
75%,300.000000,12373.000000
max,2820.000000,79512.000000


In [18]:
# formatting the stops. 


df['Total_Stops'] = df['Total_Stops'].str.split(" ").str[0]
df['Total_Stops'].replace('non-stop' , 0 , inplace = True)


# Converting the stops dtype to int
df['Total_Stops'] = df["Total_Stops"].astype(int)

<hr style="border:2px solid gray">

#**STEP: 3/5** - Split the Data & Determine the Baseline

In [19]:
# Split the data into Feature Matrix and Target Vector
target = 'Price'
X = df.select_dtypes('number').drop(columns=target)
y = df[target]

In [20]:
X.head() # 2D, DataFrame

,Duration,Total_Stops
0,170,0
1,445,2
2,1140,2
3,325,1
4,285,1


In [21]:
y.head() # 1D, Vector

0     3897
1     7662
2    13882
3     6218
4    13302
Name: Price, dtype: int64

In [22]:
# Split data into Train and Test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
print("The size of training input is", X_train.shape)
print("The size of training output is", y_train.shape)
print(50 *'*')
print("The size of testing input is", X_test.shape)
print("The size of testing output is", y_test.shape)

The size of training input is (8545, 2)
The size of training output is (8545,)
**************************************************
The size of testing input is (2137, 2)
The size of testing output is (2137,)


In [24]:
# Determine the baseline MAE
y_pred_baseline = [y_train.mean()]*len(y_train)
print('BASELINE MAE', mean_absolute_error(y_train, y_pred_baseline))

BASELINE MAE 3649.7532864424425


<hr style="border:2px solid gray">

#**STEP: 4/5** - Build & Evaluate the model 

In [25]:
# Instantiate the predictor
model = LinearRegression()

In [26]:
# Fitting the model on training data
model.fit(X_train, y_train)

LinearRegression()

In [27]:
# MAE of LinearRegression model.

print('Linear Regression TRAIN MAE',mean_absolute_error(y_train, model.predict(X_train)))
print('Linear Regression TEST MAE',mean_absolute_error(y_test, model.predict(X_test)))

Linear Regression TRAIN MAE 2553.8511093118786
Linear Regression TEST MAE 2589.0706063419348


In [28]:
# RMSE of LinearRegression model.

print('Linear Regression TRAIN RMSE',mean_squared_error(y_train, model.predict(X_train), squared=False)) # Why do we cancel the square?
print('Linear Regression TEST RMSE',mean_squared_error(y_test, model.predict(X_test), squared=False))

Linear Regression TRAIN RMSE 3656.506811439123
Linear Regression TEST RMSE 3739.368248595093


In [29]:
# r2 of LinearRegression model.

print('Linear Regression TRAIN R2',r2_score(y_train, model.predict(X_train)))
print('Linear Regression TEST R2',r2_score(y_test, model.predict(X_test)))

Linear Regression TRAIN R2 0.36902677348109614
Linear Regression TEST R2 0.35150550168388384


<hr style="border:2px solid gray">

#**STEP: 5/5** - Determine the Linear Equation used by the Model

In [30]:
# Checking the coefficient(slope) and intercepts
coefficient = model.coef_
intercept = model.intercept_

In [31]:
# final equation
print(f'Price = {intercept} + {coefficient[0]} * Duration + {coefficient[1]} * Total_Stops')


Price = 5568.444249852562 + 0.5523940155604133 * Duration + 4085.947660873711 * Total_Stops
